# Risk Flow Matrix Modeling

In [1]:
import pandas as pd
import numpy as np
import math

## Data Preprocessing

### Read the preprocessed dataset:
 - "origin_dest_trips_census_tract_level.csv"
 - "tracts_4.csv"

In [2]:
tracts = pd.read_csv('risk_flow_data/tracts_4.csv')
tracts
# tracts = tracts.drop(columns = ('Unnamed: 0'))

,GEOID,ZIP_CODES,TRIPS_ORIG,TRIPS_DEST,POPULATION,STORES,CBSA_EMP,CBSA_POP,CBSA_WRK,COUNTHU10,...,No. Populated Places,No. Stores,Total Points of Interest,POP_DENSITY,NUM_WORKERS,EMPLOYMENT_DENSITY,PREDICTED_ORIG_TRIPS,PREDICTED_DEST_TRIPS,FIPS,avg_cases_per_tract
0,17091011700,0,27.0,111.0,3417,0,43299.0,113449.0,46799.0,378.250000,...,0.0,0,8.0,7.059873,46799.0,2.442126,126.347,125.582,17091,63.068966
1,17091011800,0,93.0,141.0,2627,0,43299.0,113449.0,46799.0,589.000000,...,1.0,0,3.0,5.838794,46799.0,1.327268,116.855,155.451,17091,63.068966
2,17119400951,0,433.0,299.0,4966,0,1261547.0,2812896.0,1237055.0,749.333333,...,0.0,0,4.0,3.772428,1237055.0,0.566353,314.676,283.279,17119,46.803279
3,17119400952,0,4.0,155.0,3335,0,1261547.0,2812896.0,1237055.0,712.000000,...,1.0,0,2.0,2.777786,1237055.0,0.242828,60.534,203.733,17119,46.803279
4,17135957500,['62533' '62560'],34.0,45.0,3273,0,0.0,0.0,0.0,372.000000,...,7.0,0,49.0,0.128183,0.0,0.040990,41.602,70.198,17135,21.875000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3118,17037000100,['60135'],145.0,245.0,6712,0,4161510.0,9461105.0,4066635.0,852.666667,...,2.0,0,18.0,1.022809,4066635.0,0.274031,160.356,242.515,17037,45.714286
3119,17037001500,0,346.0,156.0,3807,0,4161510.0,9461105.0,4066635.0,551.666667,...,1.0,0,8.0,5.213170,4066635.0,1.535521,404.125,199.039,17037,45.714286
3120,17037000400,['60178'],13.0,499.0,8622,2,4161510.0,9461105.0,4066635.0,1652.500000,...,4.0,2,24.0,0.252156,4066635.0,0.009529,193.709,440.224,17037,45.714286
3121,17037000300,['60111' '60150'],110.0,59.0,2680,0,4161510.0,9461105.0,4066635.0,552.000000,...,3.0,0,33.0,0.037200,4066635.0,0.012366,87.904,65.580,17037,45.714286


In [3]:
# this dataset is for the nested hashmap
OD_census_tract_level = pd.read_csv('risk_flow_data/origin_dest_trips_census_tract_level.csv')
OD_census_tract_level

,RESIDENCE,WORKPLACE,JOBS
0,17001000100,17001000100,139
1,17001000100,17001000201,12
2,17001000100,17001000202,9
3,17001000100,17001000400,10
4,17001000100,17001000500,119
...,...,...,...
80423,17203030700,17113005102,2
80424,17203030700,17143001600,5
80425,17203030700,17179021201,2
80426,17203030700,17203030601,6


### Create a dictionary: to record each origin->destination: number of trips

use the dataset: OD_census_tract_level 

1. Generate a unique set of origins in commute trips

In [4]:
origins = OD_census_tract_level.loc[:,'RESIDENCE'].unique().tolist()
#origins

2. Generate a doubly nested hashmap (python dictionary: key-value pair): \
 first mapping: **origin -> destination** \
 second mapping: **destination -> number of trips**

In [5]:
# rename the origin-> destination df
OD = OD_census_tract_level
# initial i to 0, mRes to an ampty map
i = 0; mRes = {}
# loop through all the origins 
for origin in origins:
    mWork = {}
    # for each origins, loop through all the destinations
    while i < OD.shape[0] and OD.iloc[i,0] == origin: 
        workplace = OD.iloc[i,1]
        jobs = OD.iloc[i,2]
        mWork[workplace] = jobs
        i += 1
    mRes[origin] = mWork

In [ ]:
# the hashmap
mRes

### Precess the tracts data (GEOID, TRIPS_ORIG, CASES)

use the dataset: tracts

In [81]:
# the origianl dataset read from csv file
tracts

,GEOID,ZIP_CODES,TRIPS_ORIG,TRIPS_DEST,POPULATION,STORES,CBSA_EMP,CBSA_POP,CBSA_WRK,COUNTHU10,...,No. Populated Places,No. Stores,Total Points of Interest,POP_DENSITY,NUM_WORKERS,EMPLOYMENT_DENSITY,PREDICTED_ORIG_TRIPS,PREDICTED_DEST_TRIPS,FIPS,avg_cases_per_tract
0,17091011700,0,27.0,111.0,3417,0,43299.0,113449.0,46799.0,378.250000,...,0.0,0,8.0,7.059873,46799.0,2.442126,126.347,125.582,17091,63.068966
1,17091011800,0,93.0,141.0,2627,0,43299.0,113449.0,46799.0,589.000000,...,1.0,0,3.0,5.838794,46799.0,1.327268,116.855,155.451,17091,63.068966
2,17119400951,0,433.0,299.0,4966,0,1261547.0,2812896.0,1237055.0,749.333333,...,0.0,0,4.0,3.772428,1237055.0,0.566353,314.676,283.279,17119,46.803279
3,17119400952,0,4.0,155.0,3335,0,1261547.0,2812896.0,1237055.0,712.000000,...,1.0,0,2.0,2.777786,1237055.0,0.242828,60.534,203.733,17119,46.803279
4,17135957500,['62533' '62560'],34.0,45.0,3273,0,0.0,0.0,0.0,372.000000,...,7.0,0,49.0,0.128183,0.0,0.040990,41.602,70.198,17135,21.875000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3118,17037000100,['60135'],145.0,245.0,6712,0,4161510.0,9461105.0,4066635.0,852.666667,...,2.0,0,18.0,1.022809,4066635.0,0.274031,160.356,242.515,17037,45.714286
3119,17037001500,0,346.0,156.0,3807,0,4161510.0,9461105.0,4066635.0,551.666667,...,1.0,0,8.0,5.213170,4066635.0,1.535521,404.125,199.039,17037,45.714286
3120,17037000400,['60178'],13.0,499.0,8622,2,4161510.0,9461105.0,4066635.0,1652.500000,...,4.0,2,24.0,0.252156,4066635.0,0.009529,193.709,440.224,17037,45.714286
3121,17037000300,['60111' '60150'],110.0,59.0,2680,0,4161510.0,9461105.0,4066635.0,552.000000,...,3.0,0,33.0,0.037200,4066635.0,0.012366,87.904,65.580,17037,45.714286


In [82]:
# rename columns
tracts = tracts.rename(columns = {'avg_cases_per_tract' : 'CASES'})
# select following columns
### what is 'TRIPS_ORIG'???
tracts = tracts.loc[:,('GEOID','TRIPS_ORIG','CASES')]
# convert the type to int
tracts.loc[:,'GEOID'] = tracts.loc[:,'GEOID'].astype(int)

In [83]:
ct_info_cases_data = pd.read_csv("Statistical_Analysis/data/ct_info_cases_data.csv")
ct_cases_df = ct_info_cases_data[['GEOID','countyFIPS','estimate_cases1','estimate_cases2','estimate_cases3','estimate_cases4']]
ct_cases_df

,GEOID,countyFIPS,estimate_cases1,estimate_cases2,estimate_cases3,estimate_cases4
0,17091011700,17091,0.030119,27.107335,38.311700,116.471181
1,17091011800,17091,0.023156,20.840201,29.454151,89.543398
2,17119400951,17119,0.018442,10.548614,17.187603,142.941103
3,17119400952,17119,0.012385,7.084098,11.542621,95.994478
4,17135957500,17135,0.000000,4.240201,4.783816,74.692765
...,...,...,...,...,...,...
3118,17037000100,17037,0.000000,25.466793,35.104603,177.373983
3119,17037001500,17037,0.000000,14.444589,19.911088,100.605297
3120,17037000400,17037,0.000000,32.713750,45.094142,227.848402
3121,17037000300,17037,0.000000,10.168505,14.016736,70.822746


In [84]:
tracts = tracts.merge(ct_cases_df,on = "GEOID")
tracts

,GEOID,TRIPS_ORIG,CASES,countyFIPS,estimate_cases1,estimate_cases2,estimate_cases3,estimate_cases4
0,17091011700,27.0,63.068966,17091,0.030119,27.107335,38.311700,116.471181
1,17091011800,93.0,63.068966,17091,0.023156,20.840201,29.454151,89.543398
2,17119400951,433.0,46.803279,17119,0.018442,10.548614,17.187603,142.941103
3,17119400952,4.0,46.803279,17119,0.012385,7.084098,11.542621,95.994478
4,17135957500,34.0,21.875000,17135,0.000000,4.240201,4.783816,74.692765
...,...,...,...,...,...,...,...,...
3118,17037000100,145.0,45.714286,17037,0.000000,25.466793,35.104603,177.373983
3119,17037001500,346.0,45.714286,17037,0.000000,14.444589,19.911088,100.605297
3120,17037000400,13.0,45.714286,17037,0.000000,32.713750,45.094142,227.848402
3121,17037000300,110.0,45.714286,17037,0.000000,10.168505,14.016736,70.822746


## Build Risk Flow Model

Formula: \
risk from census tract from i to j: \
case_number_at_i * num_of_trips_from_i_to_j / sum_of_product_of_cases_at_j_and_flux_from_j_for_all_j's

The risk of importation of COVID-19 cases in a country outside China ($\alpha$), from a city in China ($i$)

$r_{i\alpha}$ is the risk flow matrix

$$ r_{i\alpha} = \frac{e_i n_i }{\Sigma_je_jn_j} P_{i\alpha}$$

- where $r_i$ is the cumulttive incidence in city $i$ 
        (assumed to be homogeneous within each province)
- $n_i$ is the travel flux from $i$
- $P_{i\alpha}$ is the probability of traveling from city $i$ to country $\alpha$



The risk of importation of COVID-19 cases in a country outside China ($\alpha$), from a city in China ($i$)

$r_{i\alpha}$ is the risk flow matrix

$$ r_{i\alpha} = \frac{e_i n_i }{\Sigma_je_jn_j} P_{i\alpha}$$

- where $r_i$ is the cumulttive incidence in city $i$ 
        (assumed to be homogeneous within each province)
- $n_i$ is the travel flux from $i$
- $P_{i\alpha}$ is the probability of traveling from city $i$ to country $\alpha$

In [99]:
# fill the missing value with 0
tracts.loc[:,'TRIPS_ORIG'] = tracts.loc[:,'TRIPS_ORIG'].fillna(0)
tracts.loc[:,'CASES'] = tracts.loc[:,'CASES'].fillna(0)

# calulate total risk flow:
# sum of the "TRIPS_ORIG" * "CASES" for each "GEOID"
total_risk_flow1 = np.sum(tracts.iloc[:, 4] * tracts.iloc[:, 1])
total_risk_flow2 = np.sum(tracts.iloc[:, 5] * tracts.iloc[:, 1])
total_risk_flow3 = np.sum(tracts.iloc[:, 6] * tracts.iloc[:, 1])
total_risk_flow4 = np.sum(tracts.iloc[:, 7] * tracts.iloc[:, 1])
print([total_risk_flow1,total_risk_flow2,total_risk_flow3,total_risk_flow4])

[128773.5252788664, 26017321.79194542, 31420898.760238957, 92596373.63828875]


In [125]:
# populate rest of the rows
risk_flow_matrix1 = []
num_geoID = tracts.shape[0]
for i in range(num_geoID):
    row = [] # each row in the risk flow matrix
    cases_at_i = tracts.iloc[i, 4]    # case number at i
    origin_at_i = tracts.iloc[i, 0] # residence
    m_work = mRes.get(origin_at_i,{}) # returns the value for the specified key or {} 
    for j in range(num_geoID):       # for each dest at j
        dest_at_j = tracts.iloc[j, 0]    # geoid at dest j
        trips = 0
        if len(m_work) != 0:
            trips = m_work.get(dest_at_j, 0) * 109.278621/100
        row.append((cases_at_i * trips/total_risk_flow1))
    risk_flow_matrix1.append(row)

In [126]:
# convert to datdframe
risk_flow_matrix1_df = pd.DataFrame(risk_flow_matrix1, index = tracts.loc[:,'GEOID'], columns = tracts.loc[:, 'GEOID'])
risk_flow_matrix1_df

GEOID,17091011700,17091011800,17119400951,17119400952,17135957500,17119401100,17119401500,17119401722,17189950200,17189950400,...,17037000900,17037001600,17037000500,17037001700,17037001900,17037000100,17037001500,17037000400,17037000300,17037000200
GEOID,,,,,,,,,,,,,,,,,,,,,
17091011700,5.111907e-07,0.000000,0.000000,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
17091011800,3.930049e-07,0.000003,0.000000,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
17119400951,0.000000e+00,0.000000,0.000020,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
17119400952,0.000000e+00,0.000000,0.000002,2.101973e-07,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
17135957500,0.000000e+00,0.000000,0.000000,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17037000100,0.000000e+00,0.000000,0.000000,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
17037001500,0.000000e+00,0.000000,0.000000,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
17037000400,0.000000e+00,0.000000,0.000000,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [127]:
# export the matrix to an excel file
risk_flow_matrix1_df.to_csv('risk_flow_matrix1.csv')

In [128]:
# populate rest of the rows
risk_flow_matrix2 = []
num_geoID = tracts.shape[0]
for i in range(num_geoID):
    row = [] # each row in the risk flow matrix
    cases_at_i = tracts.iloc[i, 5]    # case number at i
    origin_at_i = tracts.iloc[i, 0] # residence
    m_work = mRes.get(origin_at_i,{}) # returns the value for the specified key or {} 
    for j in range(num_geoID):       # for each dest at j
        dest_at_j = tracts.iloc[j, 0]    # geoid at dest j
        trips = 0
        if len(m_work) != 0:
            trips = m_work.get(dest_at_j, 0) * 73.2133333/100
        row.append((cases_at_i * trips/total_risk_flow2))
    risk_flow_matrix2.append(row)
#convert to datdframe
risk_flow_matrix2_df = pd.DataFrame(risk_flow_matrix2, index = tracts.loc[:,'GEOID'], columns = tracts.loc[:, 'GEOID'])
# export the matrix to an excel file
risk_flow_matrix2_df.to_csv('risk_flow_matrix2.csv')
risk_flow_matrix2_df

GEOID,17091011700,17091011800,17119400951,17119400952,17135957500,17119401100,17119401500,17119401722,17189950200,17189950400,...,17037000900,17037001600,17037000500,17037001700,17037001900,17037000100,17037001500,17037000400,17037000300,17037000200
GEOID,,,,,,,,,,,,,,,,,,,,,
17091011700,0.000002,0.000000,0.000000,0.000000e+00,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.000000,0.000000,0.000000,0.000000e+00,0.000000
17091011800,0.000001,0.000009,0.000000,0.000000e+00,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.000000,0.000000,0.000000,0.000000e+00,0.000000
17119400951,0.000000,0.000000,0.000038,0.000000e+00,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.000000,0.000000,0.000000,0.000000e+00,0.000000
17119400952,0.000000,0.000000,0.000004,3.986962e-07,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.000000,0.000000,0.000000,0.000000e+00,0.000000
17135957500,0.000000,0.000000,0.000000,0.000000e+00,0.000004,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.000000,0.000000,0.000000,0.000000e+00,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17037000100,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,6.449772e-06,0.000000e+00,0.0,0.0,0.000078,0.000001,0.000000,0.000000e+00,0.000003
17037001500,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,8.129480e-07,8.129480e-07,0.0,0.0,0.000000,0.000013,0.000000,8.129480e-07,0.000000
17037000400,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,3.682289e-06,5.523434e-06,0.0,0.0,0.000000,0.000010,0.000005,5.523434e-06,0.000000


In [129]:
risk_flow_matrix2_df

GEOID,17091011700,17091011800,17119400951,17119400952,17135957500,17119401100,17119401500,17119401722,17189950200,17189950400,...,17037000900,17037001600,17037000500,17037001700,17037001900,17037000100,17037001500,17037000400,17037000300,17037000200
GEOID,,,,,,,,,,,,,,,,,,,,,
17091011700,0.000002,0.000000,0.000000,0.000000e+00,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.000000,0.000000,0.000000,0.000000e+00,0.000000
17091011800,0.000001,0.000009,0.000000,0.000000e+00,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.000000,0.000000,0.000000,0.000000e+00,0.000000
17119400951,0.000000,0.000000,0.000038,0.000000e+00,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.000000,0.000000,0.000000,0.000000e+00,0.000000
17119400952,0.000000,0.000000,0.000004,3.986962e-07,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.000000,0.000000,0.000000,0.000000e+00,0.000000
17135957500,0.000000,0.000000,0.000000,0.000000e+00,0.000004,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.000000,0.000000,0.000000,0.000000e+00,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17037000100,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,6.449772e-06,0.000000e+00,0.0,0.0,0.000078,0.000001,0.000000,0.000000e+00,0.000003
17037001500,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,8.129480e-07,8.129480e-07,0.0,0.0,0.000000,0.000013,0.000000,8.129480e-07,0.000000
17037000400,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,3.682289e-06,5.523434e-06,0.0,0.0,0.000000,0.000010,0.000005,5.523434e-06,0.000000


In [130]:
### populate rest of the rows
risk_flow_matrix3 = []
num_geoID = tracts.shape[0]
for i in range(num_geoID):
    row = [] # each row in the risk flow matrix
    cases_at_i = tracts.iloc[i, 6]    # case number at i
    origin_at_i = tracts.iloc[i, 0] # residence
    m_work = mRes.get(origin_at_i,{}) # returns the value for the specified key or {} 
    for j in range(num_geoID):       # for each dest at j
        dest_at_j = tracts.iloc[j, 0]    # geoid at dest j
        trips = 0
        if len(m_work) != 0:
            trips = m_work.get(dest_at_j, 0) * 130.970323/100
        row.append((cases_at_i * trips/total_risk_flow3))
    risk_flow_matrix3.append(row)

#convert to datdframe
risk_flow_matrix3_df = pd.DataFrame(risk_flow_matrix3, index = tracts.loc[:,'GEOID'], columns = tracts.loc[:, 'GEOID'])
# export the matrix to an excel file
risk_flow_matrix3_df.to_csv('risk_flow_matrix3.csv')

In [131]:
# populate rest of the rows
risk_flow_matrix4 = []
num_geoID = tracts.shape[0]
for i in range(num_geoID):
    row = [] # each row in the risk flow matrix
    cases_at_i = tracts.iloc[i, 7]    # case number at i
    origin_at_i = tracts.iloc[i, 0] # residence
    m_work = mRes.get(origin_at_i,{}) # returns the value for the specified key or {} 
    for j in range(num_geoID):       # for each dest at j
        dest_at_j = tracts.iloc[j, 0]    # geoid at dest j
        trips = 0
        if len(m_work) != 0:
            trips = m_work.get(dest_at_j, 0) * 139.87/100
        row.append((cases_at_i * trips/total_risk_flow4))
    risk_flow_matrix4.append(row)
#convert to datdframe
risk_flow_matrix4_df = pd.DataFrame(risk_flow_matrix4, index = tracts.loc[:,'GEOID'], columns = tracts.loc[:, 'GEOID'])
# export the matrix to an excel file
risk_flow_matrix4_df.to_csv('risk_flow_matrix4.csv')

In [132]:
risk_flow_matrix3_df

GEOID,17091011700,17091011800,17119400951,17119400952,17135957500,17119401100,17119401500,17119401722,17189950200,17189950400,...,17037000900,17037001600,17037000500,17037001700,17037001900,17037000100,17037001500,17037000400,17037000300,17037000200
GEOID,,,,,,,,,,,,,,,,,,,,,
17091011700,0.000003,0.00000,0.000000,0.000000e+00,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
17091011800,0.000002,0.00002,0.000000,0.000000e+00,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
17119400951,0.000000,0.00000,0.000091,0.000000e+00,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
17119400952,0.000000,0.00000,0.000009,9.622518e-07,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
17135957500,0.000000,0.00000,0.000000,0.000000e+00,0.000007,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17037000100,0.000000,0.00000,0.000000,0.000000e+00,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000013,0.000000,0.0,0.0,0.000159,0.000003,0.000000,0.000000,0.000006
17037001500,0.000000,0.00000,0.000000,0.000000e+00,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000002,0.000002,0.0,0.0,0.000000,0.000026,0.000000,0.000002,0.000000
17037000400,0.000000,0.00000,0.000000,0.000000e+00,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000008,0.000011,0.0,0.0,0.000000,0.000021,0.000009,0.000011,0.000000


### Make the Risk flow matrix for phase 1, 2, 3, 4

In [35]:
def toRiskFlowMatrix(input_path, output_path, idx):
    # this dataset is for the nested hashmap
    OD_streetlight_df = pd.read_csv(input_path)[['RESIDENCE', 'WORKPLACE', 'JOBS']]
    origins = OD_streetlight_df.loc[:,'RESIDENCE'].unique().tolist()
    # rename the origin-> destination df
    OD = OD_streetlight_df 
    # initial i to 0, mRes to an ampty map
    i = 0; mRes = {}
    # loop through all the origins 
    for origin in origins:
        mWork = {}
        # for each origins, loop through all the destinations
        while i < OD.shape[0] and OD.iloc[i,0] == origin: 
            workplace = OD.iloc[i,1]
            jobs = OD.iloc[i,2]
            mWork[workplace] = jobs
            i += 1
        mRes[origin] = mWork
    
    # populate rest of the rows
    risk_flow_matrix = []
    num_geoID = tracts.shape[0]
    total_risk_flow = np.sum(tracts.iloc[:, idx+3] * tracts.iloc[:, 1])
    for i in range(num_geoID):
        row = [] # each row in the risk flow matrix
        cases_at_i = tracts.iloc[i, idx+3]    # case number at i
        origin_at_i = tracts.iloc[i, 0] # residence
        m_work = mRes.get(origin_at_i, {}) # returns the value for the specified key or {} 
        for j in range(num_geoID):       # for each dest at j
            dest_at_j = tracts.iloc[j, 0]    # geoid at dest j
            trips = 0
            if len(m_work) != 0:
                trips = m_work.get(dest_at_j, 0)
            row.append((cases_at_i * trips/total_risk_flow))
        risk_flow_matrix.append(row)
    #convert to datdframe
    risk_flow_matrix_df = pd.DataFrame(risk_flow_matrix, index = tracts.loc[:,'GEOID'], columns = tracts.loc[:, 'GEOID'])
    # export the matrix to an excel file
    risk_flow_matrix_df.to_csv('output_path')

In [38]:
toRiskFlowMatrix(input_path = "Streetlight_Data/clean_data/streetlight_OD_p1.csv",
                 output_path = "Streetlight_Data/clean_data/risk_flow_matrix1.csv", 
                 idx = 1)

TypeError: ufunc 'true_divide' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

In [ ]:
toRiskFlowMatrix(input_path = "Streetlight_Data/clean_data/streetlight_OD_p2.csv",
                 output_path = "Streetlight_Data/clean_data/risk_flow_matrix2.csv", 
                 idx = 2)

In [ ]:
toRiskFlowMatrix(input_path = "Streetlight_Data/clean_data/streetlight_OD_p3.csv",
                 output_path = "Streetlight_Data/clean_data/risk_flow_matrix3.csv", 
                 idx = 3)

In [ ]:
toRiskFlowMatrix(input_path = "Streetlight_Data/clean_data/streetlight_OD_p4.csv",
                 output_path = "Streetlight_Data/clean_data/risk_flow_matrix4.csv", 
                 idx = 4)

In [23]:
# this dataset is for the nested hashmap
OD_streetlight_df = pd.read_csv('risk_flow_data/origin_dest_trips_census_tract_level.csv')
origins = OD_streetlight_df.loc[:,'RESIDENCE'].unique().tolist()
OD_streetlight_df

,RESIDENCE,WORKPLACE,JOBS
0,17001000100,17001000100,139
1,17001000100,17001000201,12
2,17001000100,17001000202,9
3,17001000100,17001000400,10
4,17001000100,17001000500,119
...,...,...,...
80423,17203030700,17113005102,2
80424,17203030700,17143001600,5
80425,17203030700,17179021201,2
80426,17203030700,17203030601,6


In [33]:
# rename the origin-> destination df
OD = pd.read_csv("Streetlight_Data/clean_data/streetlight_OD_p1.csv")[['RESIDENCE', 'WORKPLACE', 'JOBS']]
origins = OD.loc[:,'RESIDENCE'].unique().tolist()
# initial i to 0, mRes to an ampty map
i = 0; mRes = {}
# loop through all the origins 
for origin in origins:
    mWork = {}
    # for each origins, loop through all the destinations
    while i < OD.shape[0] and OD.iloc[i,0] == origin: 
        workplace = OD.iloc[i,1]
        jobs = OD.iloc[i,2]
        mWork[workplace] = jobs
        i += 1
    mRes[origin] = mWork

In [42]:
sum(mRes[17001000100])

289160967806

In [40]:
idx = 1
# populate rest of the rows
risk_flow_matrix = []
num_geoID = tracts.shape[0]
total_risk_flow = np.sum(tracts.iloc[:, idx+3] * tracts.iloc[:, 1])
for i in range(num_geoID):
    row = [] # each row in the risk flow matrix
    cases_at_i = tracts.iloc[i, idx+3]    # case number at i
    origin_at_i = tracts.iloc[i, 0] # residence
    m_work = mRes.get(origin_at_i, {}) # returns the value for the specified key or {} 
    for j in range(num_geoID):       # for each dest at j
        dest_at_j = tracts.iloc[j, 0]    # geoid at dest j
        trips = 0
        if len(m_work) != 0:
            trips = m_work.get(dest_at_j, 0)
        row.append((cases_at_i * trips/total_risk_flow))
    risk_flow_matrix.append(row)
#convert to datdframe
risk_flow_matrix_df = pd.DataFrame(risk_flow_matrix, index = tracts.loc[:,'GEOID'], columns = tracts.loc[:, 'GEOID'])
# export the matrix to an excel file
risk_flow_matrix_df.to_csv('risk_flow_matrix.csv')

TypeError: ufunc 'true_divide' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''